The VideoFrameGenerator I created can be installed via pip command:

#   Testing

In [ ]:
#download results and model weights
!curl -L "Link zip file here" > GRU.zip; unzip  GRU.zip; rm GRU.zip  


In [ ]:
# Download dataset
#dataset day
%cd /content/Testset
!curl -L "Link rar file here" > Dataset.rar; unrar x Dataset.rar; rm Dataset.rar 



In [ ]:
!pip install keras-video-generators

In [ ]:
#modifica cd con percorso logs corretto
%cd /content/GRU/100_/day_cut_10frame_full_94_/
%load_ext tensorboard
%tensorboard --logdir logs
%reload_ext tensorboard

In [ ]:
%cd /content/
import os
import glob
import keras
from keras_video import VideoFrameGenerator
import numpy as np
# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('Testset/*')]
classes.sort()
# some global params
SIZE = (112, 112)
CHANNELS = 3
NBFRAME = 10
BS = 8
# pattern to get videos and classes
glob_pattern='Testset/{classname}/*.mp4'
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
train2 = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.2,
    shuffle=False,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

import keras_video.utils
valid2 = train2.get_validation_generator()

In [ ]:
import keras_video.utils
keras_video.utils.show_sample(valid2)

Accuracy e visual output

In [ ]:
#modifica cd con percosro al peso corretto (e anche in model )

import keras
from keras.models import load_model
%cd /content/GRU/100_/day_cut_10frame_full_94_/chkp/
model = load_model('weights.100-0.21.hdf5')


In [ ]:
import numpy as np


#tester = test
#tester = train
tester = valid2

#compute and print average accuracy
accuracy = model.evaluate(tester)

true_labels = []
for i in range(len(tester)):
  batch = np.argmax(tester[i][1], axis=1)
  for c in batch:
    true_labels.append(c)

print(true_labels)

#compute and print predictions
results = model.predict(tester)

predict_labels = []
result_class= np.argmax(results, axis=1)
for i in result_class:
  predict_labels.append(i)

print(predict_labels)
  

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
result_class= np.argmax(results, axis=1)
labels = ['Approach', 'ChangeLane', 'Follow', 'FreeRide']

matrix = confusion_matrix(true_labels, predict_labels, normalize='true')
print (matrix)

nc = valid2.classes_count
df_cm = pd.DataFrame(matrix, range(nc), range(nc))
# plt.figure(figsize=(10,7))
sn.set(font_scale=1) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.ylabel ('True')
plt.xlabel ('Predicted')
plt.show()